In [61]:
import pandas as pd
import psycopg2
import hashlib

In [62]:
df = pd.read_csv('nba_teams.csv', delimiter=',' )
conn = psycopg2.connect(
    dbname="nba_sql2",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()
SEASON_ID = 1

In [63]:
df['Team']

0              Atlanta Hawks
1             Boston Celtics
2              Brooklyn Nets
3          Charlotte Hornets
4              Chicago Bulls
5        Cleveland Cavaliers
6           Dallas Mavericks
7             Denver Nuggets
8            Detroit Pistons
9      Golden State Warriors
10           Houston Rockets
11            Indiana Pacers
12      Los Angeles Clippers
13        Los Angeles Lakers
14         Memphis Grizzlies
15                Miami Heat
16           Milwaukee Bucks
17    Minnesota Timberwolves
18      New Orleans Pelicans
19           New York Knicks
20     Oklahoma City Thunder
21             Orlando Magic
22       Philadelphia Sixers
23              Phoenix Suns
24    Portland Trail Blazers
25          Sacramento Kings
26         San Antonio Spurs
27           Toronto Raptors
28                 Utah Jazz
29        Washington Wizards
Name: Team, dtype: object

In [67]:
index = 1
for _, row in df.iterrows():

    try:
        # Insert into Player
        cur.execute("""
            INSERT INTO Team (team_id, name)
            VALUES (%s, %s)
        """, (index,row['Team']))

        print(f"✅ Inserted {team_name}")
        index = index+1
        
    except Exception as e:
        print(f"❌ Error inserting {row['Team']}: {e}")

conn.commit()

✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards
✅ Inserted Washington Wizards


In [71]:
df = pd.read_csv('Player_Season_stats.csv', delimiter=';' , encoding="cp1252")

# --- Keep only first entry for each player (skip TOT/MIN duplicates) ---
df = df.drop_duplicates(subset=['Player'], keep='first')

In [72]:
# --- Database connection ---
conn = psycopg2.connect(
    dbname="nba_sql2",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# --- Fixed season ID ---
SEASON_ID = 1

In [77]:
conn.rollback();

In [78]:
# --- Insert players and their season stats ---
for idx, row in df.iterrows():
    name = row['Player']
    team_id = row['Tm']
    
    # Optional: use hashed player_id
    player_id = int(hashlib.md5(name.encode()).hexdigest(), 16) % (10 ** 8)

    pts = float(row['PTS'])
    trb = float(row['TRB'])
    ast = float(row['AST'])
    fg_pct = float(row['FG%']) if not pd.isna(row['FG%']) else None
    fg3_pct = float(row['3P%']) if not pd.isna(row['3P%']) else None
    ft_pct = float(row['FT%']) if not pd.isna(row['FT%']) else None
    efg_pct = float(row['eFG%']) if not pd.isna(row['eFG%']) else None

    try:
        # Insert into Player
        cur.execute("""
            INSERT INTO Player (player_id, name, team_id)
            VALUES (%s, %s, %s)
            ON CONFLICT (player_id) DO NOTHING;
        """, (player_id, name, team_id))

        # Insert into Player_Season
        cur.execute("""
            INSERT INTO Player_Season (player_id, season_id, pts, trb, ast, fg_pct, fg3_pct, ft_pct, efg_pct)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (player_id, season_id) DO NOTHING;
        """, (player_id, SEASON_ID, pts, trb, ast, fg_pct, fg3_pct, ft_pct, efg_pct))
        
        print(f"✅ Inserted {name}")
        
    except Exception as e:
        print(f"❌ Error inserting {name}: {e}")

✅ Inserted Precious Achiuwa
✅ Inserted Steven Adams
✅ Inserted Bam Adebayo
✅ Inserted Ochai Agbaji
✅ Inserted Santi Aldama
✅ Inserted Nickeil Alexander-Walker
✅ Inserted Grayson Allen
✅ Inserted Jarrett Allen
✅ Inserted Jose Alvarado
✅ Inserted Kyle Anderson
✅ Inserted Giannis Antetokounmpo
✅ Inserted Thanasis Antetokounmpo
✅ Inserted Cole Anthony
✅ Inserted OG Anunoby
✅ Inserted Ryan Arcidiacono
✅ Inserted Deni Avdija
✅ Inserted Deandre Ayton
✅ Inserted Udoka Azubuike
✅ Inserted Marvin Bagley III
✅ Inserted Patrick Baldwin Jr.
✅ Inserted LaMelo Ball
✅ Inserted Mo Bamba
✅ Inserted Paolo Banchero
✅ Inserted Desmond Bane
✅ Inserted Dalano Banton
✅ Inserted Dominick Barlow
✅ Inserted Harrison Barnes
✅ Inserted Scottie Barnes
✅ Inserted RJ Barrett
✅ Inserted Will Barton
✅ Inserted Charles Bassey
✅ Inserted Keita Bates-Diop
✅ Inserted Nicolas Batum
✅ Inserted Darius Bazley
✅ Inserted Bradley Beal
✅ Inserted Malik Beasley
✅ Inserted MarJon Beauchamp
✅ Inserted D?vis Bert?ns
✅ Inserted Patric

In [79]:
# --- Commit and close ---
conn.commit()
cur.close()
conn.close()

In [ ]:
games_data_2022_23.csv

In [117]:
df = pd.read_csv('games_data_2022_23.csv', delimiter=',')

In [118]:
df['Home_Neutral']

0               Boston Celtics
1        Golden State Warriors
2              Detroit Pistons
3               Indiana Pacers
4                Atlanta Hawks
                 ...          
1225        Los Angeles Lakers
1226    Minnesota Timberwolves
1227     Oklahoma City Thunder
1228              Phoenix Suns
1229    Portland Trail Blazers
Name: Home_Neutral, Length: 1230, dtype: object

In [119]:
# --- Database connection ---
conn = psycopg2.connect(
    dbname="nba_sql2",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# --- Fixed season ID ---
SEASON_ID = 1

In [120]:
game_id = 22200001
for idx,row in df.iterrows():
    home_team_name = row['Home_Neutral']
    away_team_name = row['Visitor_Neutral']

    score_home_team = int(row['PTS_Home'])
    score_away_team = int(row['PTS_Visitor'])

    try:
    # Get team IDs properly (fetch result after execution)
        cur.execute("SELECT team_id FROM team WHERE name = %s;", (home_team_name,))
        home_team_id = cur.fetchone()
    
        cur.execute("SELECT team_id FROM team WHERE name = %s;", (away_team_name,))
        away_team_id = cur.fetchone()

    # If team not found
        if home_team_id is None or away_team_id is None:
            print(f"⚠️ Team not found: {home_team_name} or {away_team_name}")
            continue

        home_team_id = home_team_id[0]
        away_team_id = away_team_id[0]
        # Insert into Player_Season
        cur.execute("""
            INSERT INTO Game (season_id, game_id, home_team_id, away_team_id, home_team_name, away_team_name, score_home_team, score_away_team)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (SEASON_ID, game_id, home_team_id, away_team_id, home_team_name, away_team_name, score_home_team, score_away_team))
        
        print(f"✅ Inserted {home_team_name}")
        game_id = game_id+1

        conn.commit()
        
    except Exception as e:
        print(f"❌ Error: {e}")
        conn.rollback()

✅ Inserted Boston Celtics
✅ Inserted Golden State Warriors
✅ Inserted Detroit Pistons
✅ Inserted Indiana Pacers
✅ Inserted Atlanta Hawks
✅ Inserted Brooklyn Nets
✅ Inserted Memphis Grizzlies
✅ Inserted Miami Heat
✅ Inserted Toronto Raptors
✅ Inserted Minnesota Timberwolves
✅ Inserted San Antonio Spurs
✅ Inserted Utah Jazz
✅ Inserted Phoenix Suns
✅ Inserted Sacramento Kings
✅ Inserted Philadelphia 76ers
✅ Inserted Los Angeles Lakers
✅ Inserted Charlotte Hornets
✅ Inserted Indiana Pacers
✅ Inserted Washington Wizards
✅ Inserted Atlanta Hawks
✅ Inserted Brooklyn Nets
✅ Inserted Miami Heat
✅ Inserted New York Knicks
✅ Inserted Houston Rockets
✅ Inserted Minnesota Timberwolves
✅ Inserted Golden State Warriors
✅ Inserted Portland Trail Blazers
✅ Inserted Philadelphia 76ers
✅ Inserted Indiana Pacers
✅ Inserted Orlando Magic
✅ Inserted Chicago Bulls
✅ Inserted Miami Heat
✅ Inserted Milwaukee Bucks
✅ Inserted Dallas Mavericks
✅ Inserted Denver Nuggets
✅ Inserted Sacramento Kings
✅ Inserted Los 

In [121]:
# --- Commit and close ---
conn.commit()
cur.close()
conn.close()

In [1]:
#########################
##ubacujem player_game##
#########################

In [1]:
import time
import psycopg2
import pandas as pd

In [2]:
DB_CONFIG = {
    "dbname" : "nba_sql2",
    "user" : "postgres",
    "password" : "1234",
    "host" : "localhost",
    "port" : "5432"
}

SEASON_ID = 1

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

In [110]:
df = pd.read_csv('PlayerStatistics.csv', delimiter=',', low_memory=False)  
#filtrirat samo utakmice u sezoni koju gledamo


In [111]:
input_file = "PlayerStatistics.csv"   # Your input CSV file
output_file = "filtered_regular_season.csv"  # Output file
start_date = pd.to_datetime("2022-10-18")
end_date = pd.to_datetime("2023-04-10")

In [112]:
df["gameDate"] = pd.to_datetime(df["gameDate"], errors="coerce")

In [113]:
mask = (df["gameDate"] >= start_date) & (df["gameDate"] <= end_date)
filtered_df = df.loc[mask]

In [114]:
filtered_df.to_csv(output_file, index=False)

print(f"✅ Filtered data saved to {output_file}")
print(f"Rows before: {len(df)}, Rows after filtering: {len(filtered_df)}")

✅ Filtered data saved to filtered_regular_season.csv
Rows before: 1627438, Rows after filtering: 31172


In [3]:
df = pd.read_csv('filtered_regular_season.csv', delimiter=',')

In [4]:

for idx,row in df.iterrows():
    player_name = row['firstName'] + ' ' + row['lastName']
    
    cur.execute("SELECT player_id FROM PLAYER WHERE name = %s;", (player_name,))
    player_id = cur.fetchone()
    if player_id is None:
        continue

    game_id = row['gameId']

    pts = row['points']
    trb = row['reboundsTotal']
    ast = row['assists']
    fg_pct = row['fieldGoalsPercentage']
    fg3_pct = row['threePointersPercentage']
    ft_pct = row['freeThrowsPercentage']

    
    try:
        cur.execute("""
            INSERT INTO Player_Game (player_id, game_id, pts, trb, ast, fg_pct, fg3_pct, ft_pct)
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
        """, (player_id, game_id, pts, trb, ast, fg_pct, fg3_pct, ft_pct))

        print(f"✅ Inserted {player_id}")


    except Exception as e:
        print(f"❌ Error: {e}")
        conn.rollback()


✅ Inserted (64323306,)
✅ Inserted (13000246,)
✅ Inserted (45351420,)
✅ Inserted (98570657,)
✅ Inserted (39745290,)
✅ Inserted (33588179,)
✅ Inserted (73174557,)
✅ Inserted (17400512,)
✅ Inserted (50287861,)
✅ Inserted (64144421,)
✅ Inserted (90633997,)
✅ Inserted (10808056,)
✅ Inserted (62789163,)
✅ Inserted (12615490,)
✅ Inserted (47138379,)
✅ Inserted (78129899,)
✅ Inserted (93753467,)
✅ Inserted (83761409,)
✅ Inserted (23198814,)
✅ Inserted (97064963,)
✅ Inserted (37551458,)
✅ Inserted (81346456,)
✅ Inserted (44043725,)
✅ Inserted (45834930,)
✅ Inserted (89389103,)
✅ Inserted (11158508,)
✅ Inserted (97225529,)
✅ Inserted (70884837,)
✅ Inserted (93775213,)
✅ Inserted (40479685,)
✅ Inserted (23257304,)
✅ Inserted (78206953,)
✅ Inserted (19572900,)
✅ Inserted (91102061,)
✅ Inserted (24746424,)
✅ Inserted (63031828,)
✅ Inserted (87165950,)
✅ Inserted (77882736,)
✅ Inserted (25717690,)
✅ Inserted (19549498,)
✅ Inserted (75928133,)
✅ Inserted (81402114,)
✅ Inserted (30611725,)
✅ Inserted 

In [5]:
conn.commit()
cur.close()
conn.close()
#uneseni podaci za svaku tekmu za svakog igraca

In [ ]:
#unosenje Team_Season podataka

In [2]:
import psycopg2
import pandas as pd

In [3]:
DB_CONFIG = {
    "dbname" : "nba_sql2",
    "user" : "postgres",
    "password" : "1234",
    "host" : "localhost",
    "port" : "5432"
}

SEASON_ID = 1

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

In [4]:
df = pd.read_csv('team_season.csv', delimiter=',')

In [5]:
for idx,row in df.iterrows():
    team_name = row['Team']

    cur.execute("SELECT team_id FROM TEAM WHERE name = %s;", (team_name,))
    team_id = cur.fetchone()

    wins, losses = row['Overall'].split('-')

    try:
        cur.execute("""
            INSERT INTO Team_Season (team_id, season_id, wins, losses)
            VALUES (%s,%s,%s,%s)
        """, (team_id, SEASON_ID, wins, losses))

        print(f"✅ Inserted {team_name}")


    except Exception as e:
        print(f"❌ Error: {e}")
        conn.rollback()


# team_id , season_id , wins, losses

✅ Inserted Milwaukee Bucks
✅ Inserted Boston Celtics
✅ Inserted Philadelphia 76ers
✅ Inserted Denver Nuggets
✅ Inserted Cleveland Cavaliers
✅ Inserted Memphis Grizzlies
✅ Inserted Sacramento Kings
✅ Inserted New York Knicks
✅ Inserted Brooklyn Nets
✅ Inserted Phoenix Suns
✅ Inserted Golden State Warriors
✅ Inserted Los Angeles Clippers
✅ Inserted Miami Heat
✅ Inserted Los Angeles Lakers
✅ Inserted Minnesota Timberwolves
✅ Inserted New Orleans Pelicans
✅ Inserted Atlanta Hawks
✅ Inserted Toronto Raptors
✅ Inserted Chicago Bulls
✅ Inserted Oklahoma City Thunder
✅ Inserted Dallas Mavericks
✅ Inserted Utah Jazz
✅ Inserted Indiana Pacers
✅ Inserted Washington Wizards
✅ Inserted Orlando Magic
✅ Inserted Portland Trail Blazers
✅ Inserted Charlotte Hornets
✅ Inserted Houston Rockets
✅ Inserted San Antonio Spurs
✅ Inserted Detroit Pistons


In [6]:
conn.commit()
cur.close()
conn.close()